Often when running experiments, we have no idea how many trials we need to run. However, we can determine the least number of samples to achieve our specifications using a two-sided t-test (a statistical test for different means using the Neyman-Pearson criterion).

A t-test is formulated as 

$$
t_i = \frac{x_i - \mu}{s/\sqrt{n}}
$$
where
$$
|t_i|  > \gamma \implies \text{accept } H_1 \\
|t_i|  < \gamma \implies \text{accept } H_0 \\
$$

such that $x_i$ is the sample, $\mu$ is the population mean, $s$ is the population standard deviation, $n$ is the number of samples (or degrees of freedom), $H_1$ is the alternative hypothesis, and $H_0$ is the null hypothesis. If we assume $x \sim \mathcal{N}(\mu, s^2)$, then $t$ is distributed according to a Student-t distribution.

In the Neyman-Pearson criterion, the goal is to maximize the probability of detection $P(H_1 | H_1)$ or power $\Beta$ given a probability of false-positives $P(H_1|H_0)$ or size $\alpha$.

In [83]:
import numpy as np
from scipy.stats import nct

alpha = 0.05
beta = 0.8
delta_mean = 3.5 # This quantifies the distance between the alternative and null hypothesis (i.e. effect size)

Given these inputs, we calculate the threshold $\gamma$ using different $n$ and terminate when we reach the desired power.

In [84]:
def calcBeta(N: float, verbose: bool = False):
    # Calculate threshold for Neyman-Pearson test to ensure 
    # the probability of false alarm given P(|test_statistic| > gamma | H_0) = alpha
    gamma = nct.ppf(1 - alpha/2, N, 0.0)
    if verbose:
        print(f"Threshold for two-sided t-test: {gamma}")

    # Calculate power or probability of detection (delta_mean makes the t-distribution a non-central t-distribution)
    upper_prob = 1.0 - nct.cdf(gamma, N, delta_mean) # P(t > gamma| H_1)
    lower_prob = nct.cdf(-gamma, N, delta_mean) # P(t < gamma | H_1)
    if verbose:
        print(f"Power (or probability of detection): {upper_prob + lower_prob}")
    return upper_prob + lower_prob

Use bisection algorithm to quickly find the best $n$!

In [85]:

MAX_N = 1000
MIN_N = 1
ERROR = 1e-3
# Assume beta is smallest at MIN_N and largest at MAX_N

max_n = MAX_N
min_n = MIN_N
it = 0
while True:
    n_proposal = (max_n + min_n) / 2.0
    beta_n_proposal = calcBeta(n_proposal)
    if abs(beta_n_proposal - beta) > ERROR and abs(n_proposal - MIN_N) > ERROR and abs(n_proposal - MAX_N) > ERROR:
        if beta_n_proposal > beta:
            max_n = n_proposal
        else:
            min_n = n_proposal
    else:
        break

print(f"{n_proposal:0.3f} samples gives a test with power {beta_n_proposal:0.3f}")


5.085 samples gives a test with power 0.800
